## About this notebook:

This notebook uses Affinder napari plugin to find transformation manually.

Input:
- pathway to a directory containing segmented labels
- pathway to a directory containing data frames
- pathway to a directory containing transformations (pkl files)
- well and round (as in the names of directories) selected for corrections

Output:
- a set of transforms for the selected well with the corrected transform for a given round

Optionally you can visualize alignment (visualization uses Napari).

## Fill in info about the experiment to process

In [14]:
# details about the experiment

# pathway to a directory with segmented masks for alignment (ex. im_segmented)
path_labels = r''

# pathway to a directory with data frames (ex. df)
path_df = r''

# pathway to save transformations (ex. df)
path_save = r''

# specify which round in which well to correct manually
myWell =''
myRound =  # as named in the directory

## Prepare for processing

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import napari
from tifffile import imsave,imread
from scipy import ndimage as ndi
from skimage import transform
import matplotlib.pyplot as plt

In [15]:
def read_labels(path_labels,myWell,alignRound):
    
    labels_list = [x for x in os.listdir(os.path.join(path_labels,myWell)) if 'tif' in x]
    labels_list.sort()

    labels_list = [labels_list[0],labels_list[int(alignRound)]]

    labels_im_list = []
    for lab_im_name in labels_list:

        lab_im = plt.imread(os.path.join(path_labels,myWell,lab_im_name))
        labels_im_list.append(lab_im)

    labels_im = np.array(labels_im_list)
    
    return labels_im

In [16]:
# open df
df_path = os.path.join(path_save,f'df_{myWell}.pkl')
df = pd.read_pickle(df_path)

# check which alignRound for the selected round
alignRound = df.loc[df.nameRound == myRound,'alignRound'].tolist()[0]

# open labels and the control
labels_im = read_labels(path_labels,myWell,alignRound)

# open the list of transformations for a given well
tmat_path = os.path.join(path_save,f'tmat_{myWell}.pkl')
with open(tmat_path,'rb') as tmat_file:
    tmat = pickle.load(tmat_file)

# select the appropriate transformation
t_org = tmat[int(alignRound)]


C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\PIL\Image.py:2896: DecompressionBombWarning: Image size (100721296 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


## Use Affinder in Napari

Remember to set parameters of the plugin correctly:
- reference image
- image to transform
- 'similarity' as transformation

Information about Affinder can be found:
- https://github.com/jni/affinder
- https://www.napari-hub.org/plugins/affinder?imageModality=Fluorescence+microscopy

In [18]:
viewer = napari.Viewer()
l0 = viewer.add_image(labels_im[0], colormap='bop blue', blending='additive',name='ref')
l1 = viewer.add_image(labels_im[1], colormap='bop purple', blending='additive',name='moving')

qtwidget, widget = viewer.window.add_plugin_dock_widget(
        'affinder', 'Start affinder'
        )
widget.reference.bind(l0)
widget.moving.bind(l1)
widget()

C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\affinder\affinder.py:28: FutureWarning: Private attribute access ('Viewer._canvas_size') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  viewer.camera.zoom = np.min(viewer._canvas_size) / np.max(size)
C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\numpy\core\numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(

## Collect transformation matrix

In [19]:
# optionally - test matrix
mat = np.asarray(viewer.layers['moving'].affine)

<Image layer 'tfd_ndi' at 0x1c6a74745e0>

## Visualize transformation from the collected matrix (optional)

In [ ]:
tfd_ndi = ndi.affine_transform(labels_im[1], np.linalg.inv(mat))
viewer.add_image(tfd_ndi, colormap='bop orange', blending='additive')

## Save transformation matrix

In [20]:
def matrix_rc2xy(affine_matrix):
    swapped_cols = affine_matrix[:, [1, 0, 2]]
    swapped_rows = swapped_cols[[1, 0, 2], :]
    return swapped_rows


tmat[int(alignRound),:,:] = np.linalg.inv(matrix_rc2xy(mat))
tmat.dump(tmat_path)